In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 21391 (delta 4), reused 5 (delta 1), pack-reused 21378
Receiving objects: 100% (21391/21391), 16.79 MiB | 12.72 MiB/s, done.
Resolving deltas: 100% (15659/15659), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728       

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer ,LabelEncoder, OneHotEncoder, MinMaxScaler, OrdinalEncoder
import seaborn as sns
! pip install prince
import prince

! pip install optuna
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

Mounted at /content/drive
     |████████████████████████████████| 296kB 11.5MB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 1.1MB 21.2MB/s 
     |████████████████████████████████| 143kB 52.4MB/s 
     |████████████████████████████████| 112kB 38.4MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 481kB 56.5MB/s 
  Created wheel for alembic: filename=alembic-1.5.4-py2.py3-none-any.whl size=156314 sha256=dc22dd99a84f28d151617047250db54e53e970a06519307b72c6a124ae699e0b
  Stored in directory: /root/.cache/pip/wheels/fd/2d/ec/5a1b1e2363ed68392d292d215facf588d5448198edd8078bc1
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=344e6c90ea77310c06008cbc8f07a373ed59a17ca1d4fa35e91044b6385ca160
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-

In [3]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [41]:
def create_stratified_folds_for_regression(data_df, n_splits=5):
    """
    @param data_df: training data to split in Stratified K Folds for a continous target value
    @param n_splits: number of splits
    @return: the training data with a column with kfold id
    """
    data_df['kfold'] = -1
    # randomize the data
    data_df = data_df.sample(frac=1, random_state=42).reset_index(drop=True)
    # calculate the optimal number of bins based on log2(data_df.shape[0])
    num_bins = np.int(np.floor(1 + np.log2(len(data_df))))
    # bins value will be the equivalent of class value of target feature used by StratifiedKFold to 
    # distribute evenly the classed over each fold
    data_df.loc[:, "bins"] = pd.cut(pd.to_numeric(data_df['target'], downcast="signed"), bins=num_bins, labels=False)
    kf = StratifiedKFold(n_splits=n_splits, random_state=42)
    
    # set the fold id as a new column in the train data
    for f, (t_, v_) in enumerate(kf.split(X=data_df, y=data_df.bins.values)):
        data_df.loc[v_, 'kfold'] = f
    
    # drop the bins column (no longer needed)
    data_df = data_df.drop("bins", axis=1)
    
    return data_df

In [9]:
def cv_evaluation_regression(df, model):
    n_fold = folds
    oof = np.zeros((300000,))
    for fold in tqdm(range(n_fold)):
        val_ind = df[df.kfold == fold].index
        train = df[df.kfold != fold].reset_index(drop=True)
        valid = df[df.kfold == fold].reset_index(drop=True)

        features = [feature for feature in df.columns if feature not in ['id', 'target', 'kfold']]

        X_train = train[features]
        y_train = train['target']
        X_valid = valid[features]
        y_valid = valid['target']

        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 500, verbose=False)
        preds = model.predict(valid[features])
        mse = mean_squared_error(y_valid, preds)
        
        oof[val_ind] = preds
    
    mse_oof = mean_squared_error(oof, df['target'])
    
    print(f'LGBM Overall RMSE:{np.sqrt(mse_oof)}')
    return mse_oof

In [38]:
path = '/content/drive/Shareddrives/dacon/Playground/'
train = pd.read_csv(path + 'train.csv', index_col=['id'])
test = pd.read_csv(path + 'test.csv', index_col=['id'])
submission = pd.read_csv(path + 'sample_submission.csv', index_col=['id'])

In [47]:
train_data = train.iloc[:,:-1]
train_target = train.iloc[:, -1]
test_data = test.copy()

folds = 5

cat_features = [feature for feature in train_fold.columns if 'cat' in feature]
cont_features = [feature for feature in train_fold.columns if 'cont' in feature]

In [40]:
scaler  = QuantileTransformer(random_state=42, n_quantiles=1500)
a = pd.get_dummies(train_data[cat_features])
b = pd.get_dummies(test_data[cat_features])

missing_cols = set(a.columns) - set(b.columns)
for c in missing_cols:
  b[c] = 0
b = b[a.columns]

train_temp = train_data[cont_features]
test_temp = test_data[cont_features]
train_sc = scaler.fit_transform(train_temp)
test_sc = scaler.transform(test_temp)

train_scaled = pd.DataFrame(train_sc, index = train_temp.index, columns = train_temp.columns)
test_scaled = pd.DataFrame(test_sc, index = test_temp.index, columns = test_temp.columns)

trainset = pd.concat([train_scaled, a, train_target], axis = 1)
testset = pd.concat([test_scaled, b], axis = 1)

trainset

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cat0_A,cat0_B,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat3_C,cat3_D,cat4_A,cat4_B,cat4_C,cat4_D,cat5_A,cat5_B,cat5_C,cat5_D,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_G,cat6_H,cat6_I,cat7_A,cat7_B,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_I,cat8_A,cat8_B,cat8_C,cat8_D,cat8_E,cat8_F,cat8_G,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.940916,0.743303,0.046189,0.262185,0.346990,0.900497,0.531811,0.895649,0.932840,0.844205,0.861846,0.678347,0.944570,0.721503,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6.994023
2,0.310632,0.015017,0.343941,0.973954,0.376355,0.495759,0.274397,0.253887,0.782310,0.590147,0.669928,0.800107,0.220230,0.922358,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8.071256
3,0.830057,0.895601,0.493890,0.936372,0.409145,0.940410,0.362533,0.944315,0.888857,0.926020,0.142713,0.655557,0.930102,0.977543,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5.760456
4,0.807019,0.937972,0.082587,0.770248,0.891234,0.962538,0.726620,0.544336,0.891863,0.816634,0.480207,0.664320,0.844488,0.582012,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,7.806457
6,0.470524,0.625662,0.598749,0.447376,0.188841,0.361440,0.837483,0.415459,0.546018,0.473193,0.861935,0.618260,0.317074,0.276864,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,0.059270,0.779462,0.093144,0.586359,0.889268,0.178653,0.064763,0.508377,0.223419,0.203659,0.102613,0.374541,0.006685,0.871086,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,8.343538
499996,0.019205,0.126351,0.797717,0.158230,0.900239,0.026820,0.062324,0.688836,0.114638,0.158871,0.173308,0.264305,0.132952,0.103993,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,7.851861
499997,0.203554,0.549040,0.514314,0.557816,0.413842,0.519039,0.422331,0.599026,0.046766,0.520089,0.102704,0.249794,0.017893,0.372121,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,7.600558


In [42]:
train_fold = create_stratified_folds_for_regression(trainset)
test_fold = testset.copy()

In [43]:
train_fold

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cat0_A,cat0_B,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat3_C,cat3_D,cat4_A,cat4_B,cat4_C,cat4_D,cat5_A,cat5_B,cat5_C,cat5_D,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_G,cat6_H,cat6_I,cat7_A,cat7_B,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_I,cat8_A,cat8_B,cat8_C,cat8_D,cat8_E,cat8_F,cat8_G,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O,target,kfold
0,0.611856,0.650045,0.596165,0.544795,0.338672,0.343202,0.829022,0.007394,0.154254,0.353430,0.263890,0.252451,0.196800,0.751189,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5.733432,0
1,0.142200,0.051322,0.646229,0.015767,0.415635,0.421115,0.688943,0.167795,0.222842,0.034074,0.283146,0.218237,0.092119,0.803997,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,6.990234,0
2,0.899243,0.730201,0.927059,0.232465,0.065806,0.861197,0.213338,0.920702,0.139198,0.697477,0.448367,0.664944,0.873672,0.888458,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,8.212900,0
3,0.212012,0.132867,0.254890,0.798973,0.558017,0.783843,0.415530,0.109285,0.128630,0.189954,0.343111,0.329627,0.126646,0.801349,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8.466549,0
4,0.296914,0.761991,0.996734,0.565035,0.499770,0.081924,0.353224,0.221357,0.229484,0.144114,0.506881,0.205450,0.245983,0.311758,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,7.449541,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.029428,0.134740,0.874043,0.327701,0.035996,0.682549,0.044734,0.073178,0.315832,0.169133,0.254157,0.247655,0.418056,0.384771,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.611676,4
299996,0.155304,0.699452,0.857328,0.455128,0.075381,0.081177,0.645934,0.365528,0.656060,0.057202,0.792266,0.818312,0.622748,0.473255,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,7.253953,4
299997,0.428760,0.717996,0.344582,0.813793,0.289014,0.359252,0.969868,0.702136,0.572862,0.440457,0.910095,0.800383,0.005879,0.702315,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.979163,4
299998,0.081712,0.271911,0.954510,0.633494,0.485010,0.142765,0.457327,0.614839,0.282880,0.149914,0.318586,0.323053,0.632748,0.057375,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,6.423450,4


In [12]:
# for feature in cat_features:
#     le = OrdinalEncoder()
#     le.fit(train_fold[feature])
#     train_fold[feature] = le.transform(train_fold[feature])
#     test_fold[feature] = le.transform(test_fold[feature])
#     # train_data[feature] = train_data[feature].astype('category')
#     # test_data[feature] = test_data[feature].astype('category')

encoder = OrdinalEncoder()
encoder.fit(train_fold[cat_features])

train_fold[cat_features] = encoder.transform(train_fold[cat_features])
test_fold[cat_features] = encoder.transform(test_fold[cat_features])

scaler = QuantileTransformer(random_state=42, n_quantiles=1500)
scaler.fit(train_fold[cont_features])

train_fold[cont_features] = scaler.transform(train_fold[cont_features])
test_fold[cont_features] = scaler.transform(test_fold[cont_features])

In [48]:
train_fold

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cat0_A,cat0_B,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat3_C,cat3_D,cat4_A,cat4_B,cat4_C,cat4_D,cat5_A,cat5_B,cat5_C,cat5_D,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_G,cat6_H,cat6_I,cat7_A,cat7_B,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_I,cat8_A,cat8_B,cat8_C,cat8_D,cat8_E,cat8_F,cat8_G,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O,target,kfold
0,0.611856,0.650045,0.596165,0.544795,0.338672,0.343202,0.829022,0.007394,0.154254,0.353430,0.263890,0.252451,0.196800,0.751189,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5.733432,0
1,0.142200,0.051322,0.646229,0.015767,0.415635,0.421115,0.688943,0.167795,0.222842,0.034074,0.283146,0.218237,0.092119,0.803997,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,6.990234,0
2,0.899243,0.730201,0.927059,0.232465,0.065806,0.861197,0.213338,0.920702,0.139198,0.697477,0.448367,0.664944,0.873672,0.888458,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,8.212900,0
3,0.212012,0.132867,0.254890,0.798973,0.558017,0.783843,0.415530,0.109285,0.128630,0.189954,0.343111,0.329627,0.126646,0.801349,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8.466549,0
4,0.296914,0.761991,0.996734,0.565035,0.499770,0.081924,0.353224,0.221357,0.229484,0.144114,0.506881,0.205450,0.245983,0.311758,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,7.449541,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.029428,0.134740,0.874043,0.327701,0.035996,0.682549,0.044734,0.073178,0.315832,0.169133,0.254157,0.247655,0.418056,0.384771,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.611676,4
299996,0.155304,0.699452,0.857328,0.455128,0.075381,0.081177,0.645934,0.365528,0.656060,0.057202,0.792266,0.818312,0.622748,0.473255,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,7.253953,4
299997,0.428760,0.717996,0.344582,0.813793,0.289014,0.359252,0.969868,0.702136,0.572862,0.440457,0.910095,0.800383,0.005879,0.702315,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.979163,4
299998,0.081712,0.271911,0.954510,0.633494,0.485010,0.142765,0.457327,0.614839,0.282880,0.149914,0.318586,0.323053,0.632748,0.057375,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,6.423450,4


In [45]:
model = lgb.LGBMRegressor()
score = cv_evaluation_regression(train_fold, model)

Training until validation scores don't improve for 800 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.714044
Training until validation scores don't improve for 800 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.719639
Training until validation scores don't improve for 800 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.716001
Training until validation scores don't improve for 800 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.715982
Training until validation scores don't improve for 800 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.716237

Model Overall RMSE:0.846392752415342


In [ ]:
def objective_LGBM(trial: Trial, X, y):

  params = {} #initialize parameters
  params['random_state'] = 42
  params['learning_rate'] = 0.01
  params['max_bin'] = 36
  params['n_estimators'] = 500

  params['feature_fraction'] = trial.suggest_loguniform('feature_fraction', 0.3, 1)
  params['subsample'] = trial.suggest_loguniform('subsample', 0.3, 1)
  params['num_leaves'] = trial.suggest_int('num_leaves', 150, 300)
  params['min_data'] = trial.suggest_int('min_data', 10, 500)
  params['max_depth'] = trial.suggest_int('max_depth', 10, 25)
  params['boosting_type'] = trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss'])
  params['min_child_weight'] = trial.suggest_loguniform('min_child_weight', 0.0001, 100)
  params['max_drop'] = trial.suggest_int('max_drop', 10, 150)

  # train_data = X.iloc[:, :-1]
  # train_target = X.iloc[:,-1]
  X_train, X_valid, Y_train,  Y_valid = train_test_split(train_data, train_target, test_size=0.20, random_state=42)
  # print('#############################    BEGIN   #######################################')
  model = lgb.LGBMRegressor(**params)
  model.fit(X_train, Y_train, eval_metric = ['RMSE'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=False)
  # print('#############################    END   #######################################')

  pred = model.predict(train_data)
  # score = cv_evaluation_regression(X, model)
  score = mean_squared_error(pred, train_target)

  return score

In [ ]:
study = optuna.create_study(direction = 'minimize')
study.optimize(lambda trial : objective_LGBM(trial, train_data, train_target), n_trials=200)

[I 2021-02-11 13:35:29,700] A new study created in memory with name: no-name-77ae7d89-b997-4569-9b17-0bddd3587f98
[I 2021-02-11 13:35:32,320] Trial 0 finished with value: 0.7592936871729381 and parameters: {'feature_fraction': 0.510651324432346, 'subsample': 0.3468344414613932, 'num_leaves': 150, 'min_data': 237, 'max_depth': 4, 'boosting_type': 'gbdt', 'min_child_weight': 58.869464514989176, 'max_drop': 28}. Best is trial 0 with value: 0.7592936871729381.


[LightGBM] [Warning] min_data_in_leaf is set with min_data=131, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=131
[LightGBM] [Warning] feature_fraction is set=0.47234239247804993, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47234239247804993


[I 2021-02-11 13:35:41,822] Trial 1 finished with value: 12.145111759086852 and parameters: {'feature_fraction': 0.47234239247804993, 'subsample': 0.3942958556644128, 'num_leaves': 72, 'min_data': 131, 'max_depth': 9, 'boosting_type': 'dart', 'min_child_weight': 0.01244869042248637, 'max_drop': 52}. Best is trial 0 with value: 0.7592936871729381.


[LightGBM] [Warning] min_data_in_leaf is set with min_data=271, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=271
[LightGBM] [Warning] feature_fraction is set=0.543092058218727, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.543092058218727


[I 2021-02-11 13:35:53,012] Trial 2 finished with value: 12.142369327883873 and parameters: {'feature_fraction': 0.543092058218727, 'subsample': 0.34275329604336907, 'num_leaves': 190, 'min_data': 271, 'max_depth': 9, 'boosting_type': 'dart', 'min_child_weight': 0.0005360154721843725, 'max_drop': 27}. Best is trial 0 with value: 0.7592936871729381.


...

[I 2021-02-11 13:55:13,528] Trial 197 finished with value: 0.7362414273911408 and parameters: {'feature_fraction': 0.8176347124120298, 'subsample': 0.4760285298821177, 'num_leaves': 194, 'min_data': 40, 'max_depth': 15, 'boosting_type': 'gbdt', 'min_child_weight': 0.011060364005495461, 'max_drop': 27}. Best is trial 149 with value: 0.7359454628809973.


[LightGBM] [Warning] min_data_in_leaf is set with min_data=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] feature_fraction is set=0.8284468796790849, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8284468796790849


[I 2021-02-11 13:55:19,441] Trial 198 finished with value: 0.7358920297889062 and parameters: {'feature_fraction': 0.8284468796790849, 'subsample': 0.4272531020106922, 'num_leaves': 200, 'min_data': 20, 'max_depth': 15, 'boosting_type': 'gbdt', 'min_child_weight': 0.00370679930009214, 'max_drop': 32}. Best is trial 198 with value: 0.7358920297889062.


[LightGBM] [Warning] min_data_in_leaf is set with min_data=59, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=59
[LightGBM] [Warning] feature_fraction is set=0.8724352566712702, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8724352566712702


[I 2021-02-11 13:55:25,353] Trial 199 finished with value: 0.7364213293868114 and parameters: {'feature_fraction': 0.8724352566712702, 'subsample': 0.43060823561319944, 'num_leaves': 192, 'min_data': 59, 'max_depth': 15, 'boosting_type': 'gbdt', 'min_child_weight': 0.008797135736537745, 'max_drop': 31}. Best is trial 198 with value: 0.7358920297889062.


In [ ]:
study.optimize(lambda trial : objective_LGBM(trial, train_data, train_target), n_trials=50)

[LightGBM] [Warning] min_data_in_leaf is set with min_data=94, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] feature_fraction is set=0.8486537208658076, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8486537208658076


[I 2021-02-11 16:14:04,743] Trial 402 finished with value: 0.6825542926567775 and parameters: {'feature_fraction': 0.8486537208658076, 'subsample': 0.5323732995646961, 'num_leaves': 207, 'min_data': 94, 'max_depth': 16, 'boosting_type': 'gbdt', 'min_child_weight': 0.0004260926454688176, 'max_drop': 57}. Best is trial 402 with value: 0.6825542926567775.


[LightGBM] [Warning] min_data_in_leaf is set with min_data=95, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=95
[LightGBM] [Warning] feature_fraction is set=0.847636056539495, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.847636056539495


[I 2021-02-11 16:14:35,302] Trial 403 finished with value: 0.6825300524057457 and parameters: {'feature_fraction': 0.847636056539495, 'subsample': 0.6265605174667493, 'num_leaves': 207, 'min_data': 95, 'max_depth': 16, 'boosting_type': 'gbdt', 'min_child_weight': 0.00044226417085216433, 'max_drop': 59}. Best is trial 403 with value: 0.6825300524057457.


[LightGBM] [Warning] min_data_in_leaf is set with min_data=96, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=96
[LightGBM] [Warning] feature_fraction is set=0.8425257171774972, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8425257171774972


...

[I 2021-02-11 16:40:39,353] Trial 449 finished with value: 0.6813159601012225 and parameters: {'feature_fraction': 0.935427953598139, 'subsample': 0.7628393302387347, 'num_leaves': 220, 'min_data': 78, 'max_depth': 19, 'boosting_type': 'goss', 'min_child_weight': 0.0008437522245056126, 'max_drop': 77}. Best is trial 448 with value: 0.6813159601012225.


[LightGBM] [Warning] min_data_in_leaf is set with min_data=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] feature_fraction is set=0.9347898424864804, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9347898424864804


[I 2021-02-11 16:41:14,849] Trial 450 finished with value: 0.6815497904804156 and parameters: {'feature_fraction': 0.9347898424864804, 'subsample': 0.7587200408184216, 'num_leaves': 220, 'min_data': 76, 'max_depth': 19, 'boosting_type': 'goss', 'min_child_weight': 0.0008601799152922763, 'max_drop': 79}. Best is trial 448 with value: 0.6813159601012225.


[LightGBM] [Warning] min_data_in_leaf is set with min_data=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] feature_fraction is set=0.939633272982788, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.939633272982788


[I 2021-02-11 16:41:49,804] Trial 451 finished with value: 0.6810160279525499 and parameters: {'feature_fraction': 0.939633272982788, 'subsample': 0.7594425005365053, 'num_leaves': 221, 'min_data': 76, 'max_depth': 19, 'boosting_type': 'goss', 'min_child_weight': 0.0009160485516886867, 'max_drop': 77}. Best is trial 451 with value: 0.6810160279525499.


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
study.best_params

{'boosting_type': 'gbdt',
 'feature_fraction': 0.6102908482597347,
 'max_depth': 15,
 'max_drop': 24,
 'min_child_weight': 0.003214629150312054,
 'min_data': 21,
 'num_leaves': 200,
 'subsample': 0.39459890010621523}

In [ ]:
study.best_params

{'boosting_type': 'goss',
 'feature_fraction': 0.939633272982788,
 'max_depth': 19,
 'max_drop': 77,
 'min_child_weight': 0.0009160485516886867,
 'min_data': 76,
 'num_leaves': 221,
 'subsample': 0.7594425005365053}